<a href="https://colab.research.google.com/github/vishnuvardhanreddychallapalli/Named-Entity-Recognition/blob/master/Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install flair

In [0]:
def get_data_and_labels(filename,is_train):
    my_data=[]
    my_labels=[]
    curr_label=[]
    curr_sent=[]
    with open(filename) as f:
        for eachline in f:
            if len(eachline.strip())==0:
                #print("length of line ={} and sent={}".format(len(eachline),eachline))
                if(is_train):
                    my_data.append(" ".join(curr_sent))
                    my_labels.append(curr_label)
                else:
                    my_data.append(" ".join(curr_sent))
                curr_label=[]
                curr_sent=[]
            else:
                my_keys=eachline.strip().split("\t")
                #print(my_keys)
                if(is_train):
                    curr_sent.append(my_keys[0])
                    curr_label.append(my_keys[1])
                else:
                    curr_sent.append(my_keys[0])
    if is_train:
        return my_data,my_labels
    else:
        return my_data

In [0]:
train_data,train_labels=get_data_and_labels('train.txt',True)

In [0]:
test_data=get_data_and_labels('test.txt',False)

In [0]:
len(train_data),len(train_labels)

(3395, 3395)

In [0]:
counter=dict()
for each_list in train_labels:
  curr_list=each_list
  for each_label in each_list:
    if counter.get(each_label)==None:
      counter[each_label]=1
    else:
      counter[each_label]+=1
counter

{'B-corporation': 221,
 'B-creative-work': 140,
 'B-group': 264,
 'B-location': 548,
 'B-person': 660,
 'B-product': 142,
 'I-corporation': 46,
 'I-creative-work': 206,
 'I-group': 150,
 'I-location': 245,
 'I-person': 335,
 'I-product': 203,
 'O': 59570}

In [0]:
import numpy as np
total=0
for val in counter.values():
  total+=val
total

62730

In [0]:
maxlen = max([len(s.split(" ")) for s in train_data])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 41


In [0]:
max_len=-1
max_sent=""
for eachsent in train_data:
    if len(eachsent.split(" "))>max_len:
        max_len=len(eachsent.split(" "))
        max_sent=eachsent

In [0]:
counter.keys()

dict_keys(['O', 'B-location', 'I-location', 'B-group', 'B-corporation', 'B-person', 'B-creative-work', 'B-product', 'I-person', 'I-creative-work', 'I-corporation', 'I-group', 'I-product'])

In [0]:
from flair.data import Sentence
from flair.models import SequenceTagger
# make a sentence
sentence = Sentence(train_data[0])
# load the NER tagger
tagger = SequenceTagger.load('ner')
# run NER over sentence
tagger.predict(sentence)

2019-11-19 04:04:44,450 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll03-english/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmpp9i050ft


100%|██████████| 432197603/432197603 [00:20<00:00, 21064348.55B/s]

2019-11-19 04:05:05,444 copying /tmp/tmpp9i050ft to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2019-11-19 04:05:06,376 removing temp file /tmp/tmpp9i050ft
2019-11-19 04:05:08,007 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


[Sentence: "@paulwalk It 's the view from where I 'm living for two weeks . Empire State Building = ESB . Pretty bad storm here last evening ." - 27 Tokens]

In [0]:
print(sentence)
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence: "@paulwalk It 's the view from where I 'm living for two weeks . Empire State Building = ESB . Pretty bad storm here last evening ." - 27 Tokens
The following NER tags are found:
LOC-span [15,16,17]: "Empire State Building"
ORG-span [19]: "ESB"


In [0]:
!pip install lime

     |████████████████████████████████| 276kB 3.5MB/s 
  Created wheel for lime: filename=lime-0.1.1.36-cp36-none-any.whl size=284191 sha256=c9be3a6bd40261140b342f2615b2aa5b9367c3740333e32d863f1d36395581a5
  Stored in directory: /root/.cache/pip/wheels/a9/2f/25/4b2127822af5761dab9a27be52e175105772aebbcbc484fb95
Successfully built lime


In [0]:
!pip install eli5
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler

     |████████████████████████████████| 112kB 3.5MB/s 


Using TensorFlow backend.


In [0]:
tagger = SequenceTagger.load('ner')
file = open("output_predictions.txt","w")
for i in range(len(test_data)):
    curr_text=test_data[i]
    sentence = Sentence(curr_text)
    # load the NER tagger
    # run NER over sentence
    tagger.predict(sentence)
    #file.write("Token ={}".format(i))
    file.write("Token={}".format(i))
    print("Token={}".format(i))
    for entity in sentence.get_spans('ner'):
        file.write(str(entity)+"\n")
        print(str(entity),end=' ')
    print()
file.close()

2019-11-19 04:16:31,101 loading file /root/.flair/models/en-ner-conll03-v0.4.pt
Token=0
LOC-span [21]: "Sonmarg" 
Token=1
LOC-span [12,13]: "Waltengoo Nar" 
Token=2

Token=3

Token=4
ORG-span [14,15,16]: "Avalanche Rescue Teams" ORG-span [18]: "ART" 
Token=5
ORG-span [27]: "Defence" PER-span [30,31]: "Rajesh Kalia" 
Token=6
LOC-span [7]: "Gurez" 
Token=7
LOC-span [12]: "Gurez" 
Token=8

Token=9
PER-span [28,29]: "Rajesh Kalia" 
Token=10
MISC-span [5,6]: "Making News" 
Token=11
PER-span [2]: "Swift" LOC-span [18]: "Mahazgund" 
Token=12
ORG-span [18]: "ARTs" 
Token=13

Token=14

Token=15
LOC-span [6]: "Srinagar" 
Token=16

Token=17
ORG-span [2]: "DRDO" 
Token=18
ORG-span [6]: "DRDO" 
Token=19

Token=20
ORG-span [2,3,4,5]: "Defence Research Development Organisation" ORG-span [7]: "DRDO" 
Token=21
ORG-span [5]: "DRDO" 
Token=22
PER-span [6]: "Bhamre" LOC-span [15,16]: "Siachen Glacier" 
Token=23
LOC-span [7]: "Siachen" 
Token=24
ORG-span [2,3]: "In Navy" 
Token=25

Token=26

Token=27

Toke